In [1]:
import os
from copy import deepcopy

In [5]:
from un_wpp import *
from un_wpp import (
    _organize_variants,
    _pivot_df,
    _extract_dimension_values,
    _load_dataset,
    _keep_relevant_rows,
)
from owid import catalog
from etl.paths import DATA_DIR

In [6]:
dataset_garden = _load_dataset()
df = dataset_garden["long"]
# Keep relevant rows for explorer
df = _keep_relevant_rows(df)
# Add estimates to projections timeseries
df = _organize_variants(df)
# Pivot

Filling all gaps (variants)
Pivot table


ValueError: Index contains duplicate entries, cannot reshape

In [9]:
x = _pivot_df(df[df.metric.isin(["sex_ratio"])])

Pivot table


In [7]:
df

,location,year,metric,sex,age,variant,value
758918,World,1950,sex_ratio,none,0,records,104.639999
758919,World,1950,sex_ratio,none,5,records,104.290001
758920,World,1950,sex_ratio,none,10,records,105.010002
758921,World,1950,sex_ratio,none,15,records,103.599998
758922,World,1950,sex_ratio,none,20,records,102.580002
...,...,...,...,...,...,...,...
16908953,Wallis and Futuna,2100,deaths,male,1-4,high,0.000000
16908954,Wallis and Futuna,2100,deaths,male,1-4,low,0.000000
16908955,Wallis and Futuna,2100,deaths,female,1-4,medium,0.000000
16908956,Wallis and Futuna,2100,deaths,female,1-4,high,0.000000


In [4]:
dataset_garden_latest_dir = DATA_DIR / "garden" / "un" / "2022-07-11" / "un_wpp"
dataset_garden = catalog.Dataset(dataset_garden_latest_dir)
df = dataset_garden["long"]

In [6]:
df = df.reset_index()

In [8]:
df.dtypes

location    category
year          uint64
metric      category
sex         category
age         category
variant     category
value        float32
dtype: object

In [7]:
set(a[2] for a in df.index)

{'birth_rate',
 'births',
 'child_mortality_rate',
 'death_rate',
 'deaths',
 'dependency_ratio_child',
 'dependency_ratio_old',
 'dependency_ratio_total',
 'fertility_rate',
 'growth_natural_rate',
 'growth_rate',
 'infant_mortality_rate',
 'life_expectancy',
 'median_age',
 'net_migration',
 'net_migration_rate',
 'population',
 'population_broad',
 'population_change',
 'population_density',
 'sex_ratio'}

In [12]:
table_garden.memory_usage(deep=True) / 1e6

Index    206.351341
value     91.690900
dtype: float64

In [13]:
def keep_relevant_rows(df):
    df = df.reset_index()
    df = df[
        df.variant.isin(["estimates", "low", "medium", "high"])
        & -(
            df.metric.isin(["net_migration", "net_migration_rate"])
            & (df.location == "World")
        )
    ].reset_index(drop=True)
    return df

In [14]:
table_garden = filter_table(table_garden)

In [15]:
# Add estimates to projections timeseries
table_garden = _organize_variants(table_garden)

Filling all gaps (variants)


In [17]:
table_garden = _pivot_df(table_garden)

Pivot table


In [82]:
to_explorer(df_explorer, by_sex=True, by_age=False, localdir="output-v3", version=3)

In [19]:
by_metric = True
by_sex = True
by_age = False
by_variant = True

In [22]:
dimension_values = _extract_dimension_values(
    table_garden, by_metric, by_sex, by_age, by_variant
)

In [26]:
df = table_garden

In [31]:
for metric, sex, age, variant in dimension_values:
    short_name = f"{metric}__{sex}__{age}__{variant}"
    # Update namings for regex and filter columns
    metric = metric if metric != no_dim_keyword else ".*"
    sex = sex if sex != no_dim_keyword else ".*"
    age = age if age != no_dim_keyword else ".*"
    variant = variant if variant != no_dim_keyword else ".*"
    regex = rf"{metric}__{sex}__{age}__{variant}"
    columns = list(df.filter(regex=regex).columns)
    df_ = df[columns].dropna(how="all")
    # print(short_name, "\t", df_.shape)

In [32]:
df_

death_rate__all__all__low
location year                           
World    1950                  19.518000
         1951                  19.075001
         1952                  18.389000
         1953                  17.989000
         1954                  17.457001
...                                  ...
Vatican  2096                  13.448000
         2097                  13.531000
         2098                  13.643000
         2099                  13.728000
         2100                  13.829000

[37448 rows x 1 columns]

In [ ]:
# Add variable field (metric__[dimensions])
df = _add_variable(df)
# Pivot
df = _pivot_df(df)